In [238]:
from Error_Compare import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [249]:
activity_dir = "data/dataset/effad897-2991-4c2c-9a6a-f85a111d0e3d"
min_time = 60
max_time = np.inf
# Settings
error_type = "Percent"
lower_threshold = 0.25
upper_threshold = np.inf
# error_type = "Absolute" #
# lower_threshold = 20
# upper_threshold = np.inf
live_shift = "cross_correlation"
#live_shift = 30

activity = os.path.split(activity_dir)[-1]
clean_dfs = load_data(activity_dir, has_uncleaned = False)

key_metrics = ["breathTime", "VT"]#, "VT", "RRAvg", "instBR"]


# Load Data
raw_chest = clean_dfs["raw_slow_df"][["time","c",]]

live_b3_df = clean_dfs["live_b3_df"][key_metrics]
pp_b3_df = clean_dfs["aws_b3_df"][key_metrics]

# Only look at a subset of the data
raw_chest = raw_chest[(raw_chest["time"] > min_time) & (raw_chest["time"] < max_time)]
live_b3_df = live_b3_df[(live_b3_df["breathTime"] > min_time) & (live_b3_df["breathTime"] < max_time)]
pp_b3_df = pp_b3_df[(pp_b3_df["breathTime"] > min_time) & (pp_b3_df["breathTime"] < max_time)]

# Clean Data
pp_df, pp_cleaning_demo = run_cleaning_process(pp_b3_df[key_metrics], demo = True)
live_df = run_cleaning_process(live_b3_df[key_metrics], demo = False)

if live_shift == "cross_correlation":
    lags = {}
    for metric in key_metrics:
        if metric == "breathTime":
            continue
        lag, corr = get_corr_lag(live_df, pp_df, metric, plot = False)
        print(f"{metric} lag: {lag}")
        lags[metric] = -lag
else:
    lags = {metric:-live_shift for metric in key_metrics}

all_error_signals, all_time_in_error, all_error_indices, all_error_fraction, all_error_durations, live_df_sh = perform_error_analysis(live_df, pp_df, key_metrics, lags = lags, error_type=error_type, lower_threshold=lower_threshold, upper_threshold=upper_threshold)
# Summary
print("Activity: ", activity)
print("Shift: ", lags)
print("Error Type: ", error_type)
print("Lower Threshold: ", lower_threshold)
print("Upper Threshold: ", upper_threshold)
print("-"*20)
print("Error Fraction: ", all_error_fraction)





VT lag: 4
Activity:  effad897-2991-4c2c-9a6a-f85a111d0e3d
Shift:  {'VT': -4}
Error Type:  Percent
Lower Threshold:  0.25
Upper Threshold:  inf
--------------------
Error Fraction:  {'VT': 0.39192003331945025}
No pickled data found in activity data directory. Cleaning data...
cleaned_data folder already exists at data/dataset\cleaned_data
VT lag: 4
Activity:  effad897-2991-4c2c-9a6a-f85a111d0e3d
Shift:  {'VT': -4}
Error Type:  Percent
Lower Threshold:  0.25
Upper Threshold:  inf
--------------------
Error Fraction:  {'VT': 0.39192003331945025}


In [250]:
# Visualize Error
fig = plot_metrics_compare(raw_chest, live_df_sh, pp_df, errors_indices=all_error_indices, show = False)
fig.show()